In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_user_attributes = _hex_json.loads("{}")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"ccb0f67c-833c-42e8-a3fb-438a44871be9\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Diabetes dataset cleaning for gosdt and guesses\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In [ ]:
# import jinja2
# raw_query = """
#     select * from "diabetes.csv";
#     
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     select * from "diabetes_prediction_dataset.csv";
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
import pandas as pd

# Assuming diabetes_prediction_dataset_2 is the larger dataset
# and diabetes is the smaller dataset

# Filter out rows with NaNs in diabetes_prediction_dataset_2
filtered_prediction_dataset = diabetes_prediction_dataset_2.dropna(how="any")

# Randomly sample 768 rows from the filtered larger dataset
sampled_dataset = filtered_prediction_dataset.sample(n=768, random_state=1)

# Reset indices to ensure correct alignment when concatenating side by side
sampled_dataset = sampled_dataset.reset_index(drop=True)
diabetes = diabetes.reset_index(drop=True)

# Concatenate the two datasets side by side (columns aligned by index)
combined_dataset = pd.concat([diabetes, sampled_dataset], axis=1)

# Check the final shape and save if needed
print("Number of rows and columns in combined dataset:", combined_dataset.shape)
combined_dataset.to_csv("new_combined_data_side_by_side.csv", index=False)



Number of rows and columns in combined dataset: (768, 18)


In [ ]:
# Determine the number of rows in each dataset
rows_prediction_dataset = len(diabetes_prediction_dataset_2)
rows_diabetes_dataset = len(diabetes)

# Conditionally sample the larger dataset to match the size of the smaller
if rows_prediction_dataset > rows_diabetes_dataset:
    # Drop any rows with NaNs in the relevant columns and reset index
    sampled_dataset = (
        diabetes_prediction_dataset_2.drop(columns=["diabetes"])  # Drop the 'diabetes' column
        .dropna(how="any")  # Drop rows with any NaNs in remaining columns
        .reset_index(drop=True)  # Reset index for consistent joining
        .sample(n=rows_diabetes_dataset, random_state=1)  # Sample to match the smaller dataset
    )
    # Perform the join with diabetes on the sampled dataset
    joined_dataset = diabetes.join(sampled_dataset, lsuffix="_left", rsuffix="_right", how="inner")
else:
    # Drop rows with any NaNs in diabetes and reset index
    sampled_dataset = (
        diabetes.dropna(how="any")  # Drop rows with any NaNs
        .reset_index(drop=True)  # Reset index for consistent joining
        .sample(n=rows_prediction_dataset, random_state=1)  # Sample to match the smaller dataset
    )
    # Join with diabetes_prediction_dataset_2
    joined_dataset = sampled_dataset.join(
        diabetes_prediction_dataset_2.drop(columns=["diabetes"]),
        lsuffix="_left",
        rsuffix="_right",
        how="inner"
    )

# Check the length and save the joined dataset to CSV
print("Number of rows in joined dataset:", len(joined_dataset))
joined_dataset.to_csv("newdata.csv", index=False)
# Check how many rows remain after dropping NaNs in each dataset
filtered_prediction_dataset = diabetes_prediction_dataset_2.drop(columns=["diabetes"]).dropna(how="any")
filtered_diabetes_dataset = diabetes.dropna(how="any")

print("Rows in diabetes_prediction_dataset_2 after dropna:", len(filtered_prediction_dataset))
print("Rows in diabetes after dropna:", len(filtered_diabetes_dataset))


Number of rows in joined dataset: 5
Rows in diabetes_prediction_dataset_2 after dropna: 100000
Rows in diabetes after dropna: 768


In [ ]:

from sklearn.preprocessing import OneHotEncoder
import pandas as pd


combined_df = joined_dataset.copy()

# Find all columns with only the number 0 and 1
variables_out_of_range = [
    col
    for col in combined_df.select_dtypes(include=["number"]).columns
    if combined_df[col].dropna().isin([0, 1]).all()
]
# One hot encode the 'Pregnancies' and 'smoking_history' columns.
encoder = OneHotEncoder(sparse=False)

pregnancies_encoded = encoder.fit_transform(combined_df[["Pregnancies"]])
pregnancies_encoded_df = pd.DataFrame(
    pregnancies_encoded,
    columns=[f"Pregnancies_{int(i)}" for i in range(pregnancies_encoded.shape[1])],
)

smoking_encoded = encoder.fit_transform(combined_df[["smoking_history"]])
smoking_encoded_df = pd.DataFrame(
    smoking_encoded,
    columns=[f"smoking_{i}" for i in range(smoking_encoded.shape[1])],
)
combined_df = pd.concat(
    [
        combined_df.drop(columns=["Pregnancies", "smoking_history"]),
        pregnancies_encoded_df,
        smoking_encoded_df,
    ],
    axis=1,
)
# Binarize 'hypertension', 'heart_disease', and 'diabetes' columns
combined_df["hypertension"] = combined_df["hypertension"].apply(
    lambda x: 1 if x > 0 else 0
)
combined_df["heart_disease"] = combined_df["heart_disease"].apply(
    lambda x: 1 if x > 0 else 0
)  # Binarize 'gender' column
combined_df["gender"] = combined_df["gender"].apply(
    lambda x: 1 if x.lower() == "male" else 0
)
print(combined_df)

print(variables_out_of_range)

/home/hexuser/.cache/pypoetry/virtualenvs/python-kernel-OtKFaj5M-py3.9/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/hexuser/.cache/pypoetry/virtualenvs/python-kernel-OtKFaj5M-py3.9/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
negative_values = {
    col: combined_df[(combined_df[col] < 1) & (combined_df[col] > 0)][col].tolist()
    for col in combined_df.columns
    if combined_df[col].dtype in ["int64", "float64"]
}


# Binarize DiabetesPedigreeFunction using its mean
mean_dpf = combined_df["DiabetesPedigreeFunction"].mean()
combined_df["DiabetesPedigreeFunction"] = combined_df["DiabetesPedigreeFunction"].apply(
    lambda x: 1 if x >= mean_dpf else 0
)

# Show all other columns that are not binarized
non_binarized_columns = combined_df.drop(
    columns=["DiabetesPedigreeFunction", "Age", "BloodPressure", "BMI"]
)
print(combined_df)

In [ ]:
mean_glucose = combined_df["Glucose"].mean()
mean_blood_pressure = combined_df["BloodPressure"].mean()
mean_skin_thickness = combined_df["SkinThickness"].mean()
mean_insulin = combined_df["Insulin"].mean()
mean_bmi = combined_df["bmi"].mean()
mean_hba1c = combined_df["HbA1c_level"].mean()

combined_df["Glucose_binarized"] = (combined_df["Glucose"] > mean_glucose).astype(int)
combined_df["BloodPressure_binarized"] = (
    combined_df["BloodPressure"] > mean_blood_pressure
).astype(int)
combined_df["SkinThickness_binarized"] = (
    combined_df["SkinThickness"] > mean_skin_thickness
).astype(int)
combined_df["Insulin_binarized"] = (combined_df["Insulin"] > mean_insulin).astype(int)
combined_df["BMI_binarized"] = (combined_df["bmi"] > mean_bmi).astype(int)
combined_df["HbA1c_binarized"] = (combined_df["HbA1c_level"] > mean_hba1c).astype(int)

# Write the updated combined_df to a CSV file
combined_df.to_csv("updated_combined_df.csv", index=False)

In [ ]:
columns_to_keep = [
    "Outcome",
    "Pregnancies_0",
    "Pregnancies_1",
    "Pregnancies_2",
    "Pregnancies_3",
    "Pregnancies_4",
    "Pregnancies_5",
    "Pregnancies_6",
    "Pregnancies_7",
    "Pregnancies_8",
    "Pregnancies_9",
    "Pregnancies_10",
    "Pregnancies_11",
    "Pregnancies_12",
    "Pregnancies_13",
    "Pregnancies_14",
    "Pregnancies_15",
    "Pregnancies_16",
    "smoking_0",
    "smoking_1",
    "smoking_2",
    "smoking_3",
    "smoking_4",
    "smoking_5",
    "Glucose_binarized",
    "BloodPressure_binarized",
    "SkinThickness_binarized",
    "Insulin_binarized",
    "BMI_binarized",
    "HbA1c_binarized",
]

missing_columns = [col for col in columns_to_keep if col not in combined_df.columns]
if missing_columns:
    raise KeyError(
        f"The following columns are missing from combined_df: {missing_columns}"
    )

combined_df = combined_df[columns_to_keep]
print(combined_df)
print(combined_df)

In [ ]:
# import jinja2
# raw_query = """
#     select * from "updated_combined_df.csv";
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# Write the updated_combined_df dataset to a CSV file
columns_to_keep = [
    "Pregnancies_0",
    "Pregnancies_1",
    "Pregnancies_2",
    "Pregnancies_3",
    "Pregnancies_4",
    "Pregnancies_5",
    "Pregnancies_6",
    "Pregnancies_7",
    "Pregnancies_8",
    "Pregnancies_9",
    "Pregnancies_10",
    "Pregnancies_11",
    "Pregnancies_12",
    "Pregnancies_13",
    "Pregnancies_14",
    "Pregnancies_15",
    "Pregnancies_16",
    "smoking_0",
    "smoking_1",
    "smoking_2",
    "smoking_3",
    "smoking_4",
    "smoking_5",
    "Glucose_binarized",
    "BloodPressure_binarized",
    "SkinThickness_binarized",
    "Insulin_binarized",
    "BMI_binarized",
    "HbA1c_binarized",
    "Outcome",
]
updated_combined_df = updated_combined_df[columns_to_keep]
updated_combined_df.to_csv("updated_combined_df.csv", index=False)
updated_combined_df